In [1]:
# import external libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re

from sklearn.metrics import accuracy_score, classification_report

%matplotlib inline

C:\Users\Abhishek\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
%run scripts/features.py

In [3]:
# load training and test data set
training = pd.read_table('./data/BingHackathonTrainingData.txt', sep='\t', header=None)
test = pd.read_table('./data/BingHackathonTestData.txt', sep='\t', header=None)

training = training[range(0,  6)]
test = test[range(0, 6)]

In [4]:
training = training.rename(columns={0: 'record_id', 1: 'topic_id', 2: 'publication_year', 3: 'authors', 4: 'title', 5: 'summary'})
test = test.rename(columns={0: 'record_id', 1: 'topic_id', 2: 'publication_year', 3: 'authors', 4: 'title', 5: 'summary'})

In [5]:
y = training.topic_id

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cross_validation import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(training, y, test_size=0.2, random_state=44)

In [8]:
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(3598, 6) (900, 6) (3598L,) (900L,)


In [9]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_selection import chi2, SelectKBest

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline

In [10]:
ft = FeatureTransformer()
logistic_regression = LogisticRegression(C=10, penalty='l2')

pipeline_log_reg = Pipeline([('ft', ft), ('log_reg', logistic_regression)])

In [80]:
ft = FeatureTransformer()
svc = SVC(kernel='linear', C=3.)

pipeline_svc = Pipeline([('ft', ft),('svc', svc)])

In [11]:
# fit logistic regression model

pipeline_log_reg.fit(X_train, y_train)

<string>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Pipeline(steps=[('ft', FeatureTransformer()), ('log_reg', LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))])

In [82]:
# fit linear svc model

# pipeline_svc.fit(X_train, y_train)

In [12]:
# predictions

# predsTrain_NB = pipeline_naive_bayes.predict(X_train.toarray())
predsTrain_LR = pipeline_log_reg.predict(X_train)
# predsTrain_RF = pipeline_rf.predict(X_train)
# predsTrain_SVC = pipeline_svc.predict(X_train)

# predsTest_NB = pipeline_naive_bayes.predict(X_test.toarray())
predsTest_LR = pipeline_log_reg.predict(X_test)
# predsTest_SVC = pipeline_svc.predict(X_test)
# predsTest_RF = pipeline_rf.predict(X_test)

<string>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
# print 'Accuracy on the training examples for Naive Bayes %f ' %(accuracy_score(y_train, predsTrain_NB))
# print 'Accuracy on the test examples for Naive Bayes %f ' %(accuracy_score(y_test, predsTest_NB))

# print '\n\n'
print 'Accuracy on the training examples for Logistic Regression %f ' %(accuracy_score(y_train, predsTrain_LR))
print 'Accuracy on the test examples for Logistic Regression %f ' %(accuracy_score(y_test, predsTest_LR))

# print 'Accuracy on the training examples for Linear SVC %f ' %(accuracy_score(y_train, predsTrain_SVC))
# print 'Accuracy on the test examples for Linear SVC %f ' %(accuracy_score(y_test, predsTest_SVC))

# print '\n\n'
# print 'Accuracy on the training examples for Random Forest Classifier %f ' %(accuracy_score(y_train, predsTrain_RF))
# print 'Accuracy on the test examples for Random Forest Classifier %f ' %(accuracy_score(y_test, predsTest_RF))

Accuracy on the training examples for Logistic Regression 0.727349 
Accuracy on the test examples for Logistic Regression 0.718889 


In [14]:
print 'Classification report for training examples \n', classification_report(y_train, predsTrain_LR)
print 'Classification report for test examples \n', classification_report(y_test, predsTest_LR)

Classification report for training examples 
             precision    recall  f1-score   support

          0       0.79      0.89      0.84      1579
          1       0.67      0.79      0.73      1359
          2       0.61      0.21      0.32       660

avg / total       0.71      0.73      0.70      3598

Classification report for test examples 
             precision    recall  f1-score   support

          0       0.84      0.84      0.84       399
          1       0.64      0.83      0.73       360
          2       0.34      0.09      0.14       141

avg / total       0.68      0.72      0.68       900



## Submission

In [15]:
# train on full training set
pipeline_log_reg.fit(training, y)

Pipeline(steps=[('ft', FeatureTransformer()), ('log_reg', LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))])

In [16]:
# predictions
preds = pipeline_log_reg.predict(test)

In [18]:
submission_df = pd.DataFrame({'record_id': test.record_id.values, 'topic_id': preds})

In [22]:
submission_df.to_csv('submissions/first.csv', sep='\t', index=False)

In [88]:
all_authors_list = []

def all_authors(authors_text):
    authors = authors_text.split(';')
    all_authors_list.extend(authors)

training.authors.map(all_authors)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
4468    None
4469    None
4470    None
4471    None
4472    None
4473    None
4474    None
4475    None
4476    None
4477    None
4478    None
4479    None
4480    None
4481    None
4482    None
4483    None
4484    None
4485    None
4486    None
4487    None
4488    None
4489    None
4490    None
4491    None
4492    None
4493    None
4494    None
4495    None
4496    None
4497    None
Name: authors, dtype: object

In [89]:
all_authors_list

['10723 10724',
 '7648 8755 10725',
 '13677 13678',
 '13679 13680',
 '9809 5003 9120',
 '1483 1319',
 '2803 6006',
 '6009 6010',
 '707 708',
 '8609 23978',
 '18555 23979',
 '22257 22258',
 '10452 9243',
 '5228 10681',
 '1441 9975',
 '5687 5688',
 '3191 14620',
 '3528 3726',
 '25518 25519',
 '1003 25520',
 '4292 25521',
 '15352 623',
 '9130 9131 9132',
 '5904 1319',
 '627 3308',
 '6865 1890',
 '7597 623',
 '576 482 1747',
 '27068 1727 27069',
 '14111 9736',
 '14520 27070',
 '6046 20574',
 '1668 3222',
 '16945 1326',
 '9156 2484',
 '6865 1890',
 '5228 625',
 '4740 16096',
 '4313 13422',
 '819 13423',
 '3856 3857',
 '3349 24975',
 '1812 10416',
 '11262 8319',
 '28782 28783 28784',
 '28785 28786 28787',
 '1111 17413',
 '3541 24046',
 '24047 24048',
 '10168 10169',
 '17175 3381',
 '2144 625',
 '2147 17174',
 '6197 1321',
 '821 17173',
 '4041 3579',
 '4042 4043',
 '12795 309',
 '1323 1324',
 '1318 1319',
 '1320 1747',
 '19701 22829',
 '9558 9852',
 '1005 1006',
 '25025 25026',
 '20707 25027'

In [ ]:
authorList = []

def create_author_list(author_text):
    author_list = author_text.split(';')
    author_list_index = [all_authors_list.index(a) for a in author_list]
    
    authorList.append(author_list_index)

map(create_author_list, training.authors)